In [1]:
    import requests
    from bs4 import BeautifulSoup

    base_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136900&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=highest&page={}"
    base_url2 = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136900&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=lowest&page={}"
    reviews = []
    for n in range(500):
        url = base_url.format(n+1)
        resp = requests.get(url)
        html = BeautifulSoup(resp.content, 'html.parser')

        score_result = html.find('div', {'class': 'score_result'})
        score_result2 = html.find('div', {'class': 'star_score'})
    
    
        lis = score_result.findAll('li')
    
        for li in lis :
            review_text = li.find('p').getText()
            star = score_result2.getText()
            comment = review_text.strip().replace("\t","").replace("\n","").replace("\r","")
            star = int(star)
            label = 0 if star < 7 else 1
            reviews.append([label, comment])
    train_data = [[], []]    
    for review in reviews:
        label = review[0]
        comment = review[1]
        
        train_data[0].append(label)
        train_data[1].append(comment)
        
    reviews = []
    for m in range(500):
        url2 = base_url2.format(n+1)
        resp = requests.get(url2)
        html = BeautifulSoup(resp.content, 'html.parser')

        score_result = html.find('div', {'class': 'score_result'})
        score_result2 = html.find('div', {'class': 'star_score'})
    
    
        lis = score_result.findAll('li')
    
        for li in lis :
            review_text = li.find('p').getText()
            star = score_result2.getText()
            comment = review_text.strip().replace("\t","").replace("\n","").replace("\r","")
            star = int(star)
            label = 0 if star < 7 else 1
            reviews.append([label, comment])    
    for review in reviews:
        label = review[0]
        comment = review[1]
        
        train_data[0].append(label)
        train_data[1].append(comment)

In [2]:
import random
combine = list(zip(train_data[0],train_data[1]))

random.shuffle(combine)

train_data[0], train_data[1] = zip(*combine)
print(train_data[0][1])
print(train_data[1][1])

1
관람객완전 재미져요 웅장함...


In [3]:
    import requests
    from bs4 import BeautifulSoup

    base_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136900&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}"
    reviews = []
    for n in range(2400,2490):
        url = base_url.format(n+1)
        resp = requests.get(url)
        html = BeautifulSoup(resp.content, 'html.parser')

        score_result = html.find('div', {'class': 'score_result'})
        score_result2 = html.find('div', {'class': 'star_score'})
    
    
        lis = score_result.findAll('li')
    
        for li in lis :
            review_text = li.find('p').getText()
            star = score_result2.getText()
            comment = review_text.strip().replace("\t","").replace("\n","").replace("\r","")
            star = int(star)
            label = 0 if star < 7 else 1
            reviews.append([label, comment])
    test_data = [[], []]    
    for review in reviews:
        label = review[0]
        comment = review[1]
        
        test_data[0].append(label)
        test_data[1].append(comment)


In [1]:
from konlpy.tag import Okt
import json
import os
from pprint import pprint

okt = Okt()

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('train_docs.json'):
    with open('train_docs.json', encoding ='UTF-8') as f:
        train_docs = json.load(f)
    with open('test_docs.json',encoding='UTF-8') as f:
        test_docs = json.load(f)
else:
    train_docs = []
    test_docs = []
    for i in range(0,len(train_data[0])):
        train_docs.append([tokenize(train_data[1][i]), train_data[0][i]])
    for i in range(0,len(test_data[0])):
        test_docs.append([tokenize(test_data[1][i]), test_data[0][i]])
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

In [2]:
from keras.preprocessing.text import Tokenizer

train_x = [d for d, _ in train_docs]
test_x = [d for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(train_x)
X_train = tokenizer.texts_to_sequences(train_x)
X_test = tokenizer.texts_to_sequences(test_x)

X_train[2]


Using TensorFlow backend.


[20, 23, 40, 155, 141, 99, 68, 56, 20, 5, 60, 34, 2]

In [3]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = len(tokenizer.word_index)+1
maxlen= 20
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
X_train[5999]
print(len(X_train))
print(len(X_test))
print(len(train_y))
print(len(test_y))
X_test[1]

10000
900
10000
900


array([  3, 189,  15, 254,  24, 217,  15,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Embedding(1000,32))
model.add(layers.LSTM(32))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
hist = model.fit(X_train, train_y, 
                   epochs=10,
                   batch_size=128,
                   validation_split=0.2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 6s 731us/sample - loss: 0.1269 - acc: 0.9862 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 2/10
8000/8000 [==============================] - 3s 382us/sample - loss: 4.3259e-04 - acc: 1.0000 - val_loss: 1.3923e-04 - val_acc: 1.0000
Epoch 3/10
8000/8000 [==============================] - 3s 384us/sample - loss: 7.2147e-05 - acc: 1.0000 - val_loss: 3.0287e-05 - val_acc: 1.0000
Epoch 4/10
8000/8000 [==============================] - 3s 395us/sample - loss: 1.5728e-05 - acc: 1.0000 - val_loss: 6.5296e-06 - val_acc: 1.0000
Epoch 5/10
8000/8000 [========================